# VAE

## autoencoder
![autoencoder](./images/autoencoder.jpg)

自编码器 (Autoencoder)类似于一个非线性的PCA，是一个利用神经网络来给复杂数据降维的模型。
- 编码器：将输入数据编码成潜在空间的表示 $z=g(X)$
- 解码器：将潜在空间的表示解码成输出数据 $\hat{x}=f(z)$
- 损失函数：$L(X, \hat{X}) = ||X - \hat{X}||^2$

假设输入$X \in R^{C \times H \times W}$是一些图片，编码器将图片编码成$z \in R^D$，解码器利用$z$将将输入重建为$\hat{X} \in R^{C \times H \times W}$。

在Autoencoder的基础上，显性的对$z$的分布进行建模，使得自编码器成为一个合格的生成模型，我们就得到了Variational Autoencoders。

## VAE (Variational Autoencoders)
对于自编码器来说，$z$的分布是不确定的，因此只能在$R^D$上采样碰运气。为什么不给定$z$一个简单的分布，将采样的空间缩的很小呢？

假设$z ～ N(0, I)$，$I$代表一个单位矩阵，即$z$是一个服从标准多元高斯分布的多维随机变量。

### Decoder
![](./images//vae-decoder.jpg)

给Decoder输入一个从$N(0, I)$中采样的$z_i$，其实是希望由$\theta$参数化的Decoder能够学会一个映射，输出对应的$X$的分布，即$p_\theta(X|z_i)$。假设，给定任意$z_i$后，$X$都服从某个各维度独立的多元高斯分布，即：
$$ p_\theta(X|z_i) = \mathcal{N}(X|\mu_i^\prime(z_i;\theta), \sigma_i^\prime(z_i;\theta)^2 * I) $$
这样一来，我们只需要输入$z_i$给Decoder，然后让它拟合出$\mu_i^\prime$和$\sigma_i^\prime$，就能知道$X|z_i$的分布了。

### Encoder
在Encoder中利用后验分布。假设有后验分布$p_\theta(z|X)$，每次前向传播时，先将$x_i$喂给Encoder，算出$z|x_i$的分布；然后再从这个分布中采样一个$z_i$，喂给Decoder，得到$X|z_i$的分布；最后基于MLE优化模型。

应用变分贝叶斯算法了！我们不妨令由$\phi$参数化的Encoder去拟合对任意$x_i$的分布，$q_\phi(z|x_i)$，我们希望这个分布能够尽可能的逼近真实的后验分布$p_\theta(z|X)$。如果$q_\phi(z|x_i)$能够足够逼近真实的后验分布的话，我们就可以直接通过Encoder得到$z|x_i$的分布了。

令近似后验分布对任意$x_i$都有
$$q_\phi(z|x_i)=N(z|\mu(x_i;\phi),\sigma^2(x_i;\phi)*I)$$
即它也是一个各维度独立的多元高斯分布。



![](./images/vae-arch.jpg)

### Loss Function
VAE的目标是极大似然估计，即最大化数据的似然概率$\log p_\theta(X)$

$$ log p_\theta(X) = \int_z q_\phi(z|X) \log p_\theta(X) dz \quad 全概率定理 $$
$$ = \int_z q_\phi(z|X) \log \frac{p_\theta(X, z)}{p_\theta(z|X)} dz \quad 贝叶斯定理 $$
$$ = \int_z q_\phi(z|X) \log \left(\frac{p_\theta(X, z)}{q_\phi(z|X)} \cdot \frac{q_\phi(z|X)}{p_\theta(z|X)}\right) dz $$
$$ = \int_z q_\phi(z|X) \log \frac{p_\theta(X, z)}{q_\phi(z|X)} dz + \int_z q_\phi(z|X) \log \frac{q_\phi(z|X)}{p_\theta(z|X)} dz $$
$$ = \ell(p_\theta, q_\phi) + D_{KL}(q_\phi, p_\theta) $$
$$\ge \ell(p_\theta, q_\phi) \quad KL散度非负 $$

显然上式中$\ell(p_\theta, q_\phi)$是log pθ(X)的一个下界，也因此我们称$\ell(p_\theta, q_\phi)$为ELBO (Evidence Lower Bound)。继续展开
$$ \ell(p_\theta, q_\phi) = \int_z q_\phi(z|X) \log \frac{p_\theta(X, z)}{q_\phi(z|X)} dz $$
$$ = \int_z q_\phi(z|X) \log \frac{p_\theta(X|z) p(z)}{q_\phi(z|X)} dz \quad 贝叶斯定理 $$
$$ = \int_z q_\phi(z|X) \log \frac{p(z)}{q_\phi(z|X)} dz + \int_z q_\phi(z|X) \log p_\theta(X|z) dz $$
$$ = -D_{KL}(q_\phi, p) + \mathbb{E}_{q_\phi}[\log p_\theta(X|z)] $$

# 一些统计知识

## 贝叶斯统计
贝叶斯公式
$$ P(\theta|X) = \frac{P(X|\theta)P(\theta)}{P(X)} $$
$\theta$ 代表一种假设（参数），$X$ 代表观测到的数据

## 先验与后验
- 先验：在观测到数据之前，对参数的分布的假设，即 $P(\theta)$
- 后验：根据观察到的样本修正之后的参数分布，即 $P(\theta|X)$
- 似然：在给定参数的情况下，观测到数据的概率，即 $P(X|\theta)$

## 交叉熵
交叉熵是信息论中的概念，用来衡量两个分布之间的差异。假设有两个分布 $P$ 和 $Q$，交叉熵定义为
$$ H(P, Q) = E_{x\sim P}[-\log Q(x)] $$

## KL散度
对于离散随机变量，分布 $P$ 和 $Q$ 之间的KL散度定义为
$$ D_{KL}(P||Q) = \sum_x P(x) \log \frac{P(x)}{Q(x)} $$

## MLE与MAP

### MLE (Maximum Likelihood Estimation)
假设有一个样本集$X=\{x_1, x_2, ..., x_N\}$，我们希望找到一个参数$\theta$，使得$X$的似然概率$p(X|\theta)$最大。

当$X$为离散型随机变量时，似然函数为
$$L(\theta) = P(X|\theta) = \prod_{i=1}^N P(x_i|\theta)$$

$$\theta_{MLE} = \arg\max_\theta logL(\theta|X) = \arg\max_\theta \sum_{i=1}^N logP(x_i|\theta)$$

### MAP (Maximum A Posteriori)
MAP是一种在估计中嵌入先验知识的方法。具体来说，若我们假设$\theta$也是一个随机变量，同时它服从某个先验分布，我们可以令
$$\theta_{MAP} = \arg\max_\theta P(\theta|X) = \arg\max_\theta \frac{p(X|\theta)p(\theta)}{p(X)}$$
也就是说，给定$\theta$的先验分布和$X$后，我们希望找到在这个情况下概率最大的取值$\theta_{MAP}$。

$$\theta_{MAP} = \arg\max_\theta logP(\theta|X) = \arg\max_\theta (logP(X|\theta) + logP(\theta)) $$

$P(X)$与$\theta$无关，所以可以忽略掉。



## EM算法
EM算法是在给定观测数据$X$的情况下，求解参数$\theta$的一种迭代算法，且包含隐变量$Z$的情况下。基本思想是：在每次迭代中，先求期望，然后求极大。

似然函数
$$ L(\theta) = \sum_{i=1}^n logP(X_i|\theta) = \sum_{i=1}^n log\sum_{j=1}^k P(X_i,Z_j|\theta) = \sum_{i=1}^n log\sum_{j=1}^k Q(Z_j)\frac{P(X_i,Z_j|\theta)}{Q(Z_j)} >= \sum_{i=1}^n \sum_{j=1}^k Q(Z_j)log\frac{P(X_i,Z_j|\theta)}{Q(Z_j)} $$
其中 $Z$ 是隐变量，$P(X_i,Z_j|\theta)$ 是完全数据的概率分布，在$Q(Z_j)=P(Z_j|X_i,\theta)$时取等号

- E步：求期望
$$ Q(\theta,\theta^{(t)}) = \sum_{Z} P(Z|X,\theta^{(t)})logP(X,Z|\theta) = E_{Z|X,\theta^{(t)}}[logP(X,Z|\theta)] $$

- M步：求极大
$$ \theta^{(t+1)} = argmax_{\theta}Q(\theta,\theta^{(t)}) $$

